In [6]:
%pip install neo4j openai langchain tiktoken langchain_openai langchainhub langchain_community py2neo

from langchain_community.graphs import Neo4jGraph
from py2neo import Graph

url = "neo4j+s://b26b71fa.databases.neo4j.io"
user = "neo4j"
password = "YWpXBbSXHIcloDAhR_ZfTo9iR6jLzFIwyux_bNBU6lA"

graph = Graph("neo4j+s://b26b71fa.databases.neo4j.io", auth=("neo4j", "YWpXBbSXHIcloDAhR_ZfTo9iR6jLzFIwyux_bNBU6lA"))

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests

url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(url).json()['query']
print(import_query)

CREATE (catalog:Microservice {name: 'CatalogService', technology: 'Java'}), (order:Microservice {name: 'OrderService', technology: 'Python'}), (user:Microservice {name: 'UserService', technology: 'Go'}), (payment:Microservice {name: 'PaymentService', technology: 'Node.js'}), (inventory:Microservice {name: 'InventoryService', technology: 'Java'}), (shipping:Microservice {name: 'ShippingService', technology: 'Python'}), (review:Microservice {name: 'ReviewService', technology: 'Go'}), (recommendation:Microservice {name: 'RecommendationService', technology: 'Node.js'}), (auth:Microservice {name: 'AuthService', technology: 'Node.js'}), (db:Dependency {name: 'Database', type: 'SQL'}), (cache:Dependency {name: 'Cache', type: 'In-memory'}), (mq:Dependency {name: 'MessageQueue', type: 'Pub-Sub'}), (api:Dependency {name: 'ExternalAPI', type: 'REST'}), (bugFixCatalog:Task {name: 'BugFix', description: 'Fix bug in CatalogService', status: 'Open'}), (featureAddOrder:Task {name: 'FeatureAdd', descri

In [8]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from py2neo import Graph
from sentence_transformers import SentenceTransformer
from transformers import GPTNeoForCausalLM, AutoTokenizer

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
gpt_neo = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

vector_index = Neo4jVector.from_existing_graph(
    hf_embeddings,
    url="neo4j+s://b26b71fa.databases.neo4j.io",
    username="neo4j",
    password="YWpXBbSXHIcloDAhR_ZfTo9iR6jLzFIwyux_bNBU6lA",
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

c:\Users\prave\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
response = vector_index.similarity_search(
    "ReviewFix",
)
print(response[0].page_content)


name: ReviewFix
description: Fix ReviewService
status: Open


In [11]:
import torch
from langchain.chains import RetrievalQA
# from langchain.vectorstores import FIASS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain import PromptTemplate
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
gpt_neo = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

generator_pipeline = pipeline(
    "text-generation", 
    model=gpt_neo, 
    tokenizer=tokenizer,
    max_length=200,
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=generator_pipeline)

vector_qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)

resonse = vector_qa.invoke(
    {"query": "What is the status of ReviewFix?"}
)

print("response")
print(response)


c:\Users\prave\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=50) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/te

TypeError: can only concatenate str (not "Document") to str